<a id='notebook_setup'></a>
# Notebook Setup

#### Table of Contents

[Table of Contents](#table_of_contents). My final report is also found here. 
Please be advised, I recommend running all cells before the table of contents can be utilized to its fullest.

In [1]:
# Dependencies and Setup
import pandas as pd

#For pd.cut function returning unneeded warnings 
pd.options.mode.chained_assignment = None  # default='warn'

#csv files 
school_csv= "Resources/schools_complete.csv"
student_csv = "Resources/students_complete.csv"

In [2]:
# Turn csv files into dataframes and look at them. Remove # from school_data.head() to see first 5 rows of school_data.
school_data = pd.read_csv(school_csv)
#school_data.head() 

In [3]:
# I don't do .head() here because I want to see what the head and tail both look like. The csv for school_data is 
# a lot shorter and concise so I get the information about its endpoint from just looking at it, unlike the student.

#Remove # from student_data to see. This time it will be first 5 rows and last 5 rows. 

student_data = pd.read_csv(student_csv)
#student_data

In [4]:
# Merge the dataframes  
district = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])


<a id='district_summary'></a>
# District Summary

A quick summary overview of the current standing of the entire district. Includes the total student amount, the total budget, and analysis on test scores across the districts math and reading scores.

Return to [Table of Contents](#table_of_contents)

In [5]:
#Creates a boolean summary of reading and math scores based on the condition greater than or equal to a passing score of 70
passing = student_data[ ["reading_score","math_score"] ] >=70

#Uses that boolean summary to create an overall summary in order to get our overall percentage of passing students.
overall = passing["reading_score"].isin([True]) & passing["math_score"].isin([True])

In [6]:
#Since everything being inputed is from a formula that will return a single value a DataFrame can be constructed without
# any external intervention such as merging.  
dist_summary_df = pd.DataFrame({
    "Total Schools":[school_data["School ID"].count()],
    "Total Students":[student_data["Student ID"].count()],
    "Total Budget":[school_data["budget"].sum()],
    "Average Math Score":[student_data["math_score"].mean()],
    "Average Reading Score":[student_data["reading_score"].mean()],
    "% Passing Math":[passing["math_score"].isin([True]).sum()/student_data["Student ID"].count()*100],
    "% Passing Reading":[passing["reading_score"].isin([True]).sum()/student_data["Student ID"].count()*100],
    "% Overall Passing": [overall.isin([True]).sum()/student_data["Student ID"].count()*100],  
})

In [7]:
dist_summary_final = dist_summary_df

In [8]:
dist_summary_final["Total Students"] = dist_summary_final["Total Students"].map("{:,.0f}".format)
dist_summary_final["Total Budget"] = dist_summary_final["Total Budget"].map("${:,.0f}".format)
dist_summary_final["Average Math Score"] = dist_summary_final["Average Math Score"].map("{:,.2f}".format)
dist_summary_final["Average Reading Score"] = dist_summary_final["Average Reading Score"].map("{:,.2f}".format)
dist_summary_final["% Passing Math"] = dist_summary_final["% Passing Math"].map("{:.2f}%".format)
dist_summary_final["% Passing Reading"] = dist_summary_final["% Passing Reading"].map("{:,.2f}%".format)
dist_summary_final["% Overall Passing"] = dist_summary_final["% Overall Passing"].map("{:,.2f}%".format)

<a id='district_summary_df'></a>
### District Summary DataFrame

In [9]:
dist_summary_final

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,15,"39,170","$24,649,428",78.99,81.88,74.98%,85.81%,65.17%


<a id='school_summary'></a>
# School Summary

Creates a DataFrame that contains a profile on each schools average performance in math and reading, and the percentages of students passing math and reading. The summary also contains the overall percentage of students passing, which is just the combination of students passing both Math and Reading. Finally, the summary also covers the type of school, the schools total alloted budget, its total student population, and how much the school spends per student.

Return to [Table of Contents](#table_of_contents)

In [10]:
#Creates the first part of school summary table, up to the budget per student. Since the Average scores are more involved and
# are mismatched in DataFrame length they will be merged in later from their own DataFrames after calculation.

s_summary_step1 = school_data[["school_name","type","size","budget"]]
s_summary_step1 = s_summary_step1.set_index("school_name")
s_summary_step1.index.name = None
s_summary_step1.sort_index(inplace=True)
s_summary_step1["Budget Per Student"] = s_summary_step1["budget"]/s_summary_step1["size"]

In [11]:
#Takes just the relevant information for getting the school averages of math scores and reading scores and groups them
# by their school name, from which their mean is taken. This is the average score.

school_scores = district[["school_name","math_score","reading_score"]]
school_avg_group = school_scores.groupby(["school_name"])
school_avg = school_avg_group.mean()

In [12]:
#The averages are then added to the table. 
s_summary_step2 = pd.concat([s_summary_step1, school_avg], axis=1)

In [13]:
#Creates the reading scores from the school_scores made above. Drops the math_scores since they interfere with counts.
# This block is seperate from the other for debugging reasons, otherwise it's entirely related with the one below it. 

read_score = school_scores.loc[(school_scores["reading_score"] >= 70),:]
schools_reading = read_score.drop(columns=["math_score"]) 
schools_reading = schools_reading.reset_index(drop=True)

In [14]:
#Completes the % passing reading started above. 

reading_group = schools_reading.groupby(["school_name"])
read_per = reading_group.count()
read_per["% Passing Reading"] = read_per["reading_score"]/s_summary_step2["size"] * 100

In [15]:
#The next two cells are the same operation as above but for the math scores. 

math_score = school_scores.loc[(school_scores["math_score"] >= 70)]
schools_math = math_score.drop(columns=["reading_score"]) 
schools_math = schools_math.reset_index(drop=True)

In [16]:
#Completes the % passing math started above. 

math_group = schools_math.groupby(["school_name"])
math_per = math_group.count()
math_per["% Passing Math"] = math_per["math_score"]/s_summary_step2["size"] * 100

In [17]:
#The next two cells are for the overall % passing and its the same as above just with the & operator which will only find 
# occurances of reading and math scores above 70. 

passing_schools = school_scores.loc[
    (school_scores["reading_score"] >= 70) &
    (school_scores["math_score"] >=70),:]

In [18]:
#completes the overall % passing started above. Again these cells are seperated for debugging reasons. 

total_passing_group = passing_schools.groupby(["school_name"])
schools_passing = total_passing_group[ ["math_score", "reading_score"] ].count()
schools_passing["% Overall Passing"] = schools_passing["math_score"]/s_summary_step2["size"] * 100

In [19]:
#Finally, the percentages found above are merged and all the data has been filled in for the school summary table. 
# Each merge gets its own cell for debugging reasons. 

In [20]:
percent_passing = pd.merge(math_per["% Passing Math"], read_per["% Passing Reading"], left_index=True, right_index=True)

In [21]:
total_passing = pd.merge(percent_passing, schools_passing["% Overall Passing"],left_index=True, right_index=True)

In [22]:
district_school_summary = pd.merge(s_summary_step2,total_passing,left_index=True, right_index=True)

In [23]:
#Now that everything has been merged together into one dataframe, I'll create a seperate DataFrame so one can be "beautified" 
# for presentation and one can live on to provide data for some specific data inspection below.

dist_school_summary = district_school_summary

dist_school_summary = dist_school_summary.rename(columns={
                                                            "type":"School Type",
                                                            "size":"Total Students",
                                                            "budget": "Total School Budget",
                                                            "math_score": "Average Math Score",
                                                            "reading_score":"Average Reading Score"
})

In [24]:
#The beautification block. Purely for show, since some of the data is inaccessible for calculation after running this. 
dist_school_summary["Total School Budget"] = dist_school_summary["Total School Budget"].map("${:,.0f}".format)
dist_school_summary["Budget Per Student"] = dist_school_summary["Budget Per Student"].map("${:,.0f}".format)
dist_school_summary["Average Math Score"] = dist_school_summary["Average Math Score"].map("{:,.2f}".format)
dist_school_summary["Average Reading Score"] = dist_school_summary["Average Reading Score"].map("{:,.2f}".format)
dist_school_summary["% Passing Math"] = dist_school_summary["% Passing Math"].map("{:.2f}%".format)
dist_school_summary["% Passing Reading"] = dist_school_summary["% Passing Reading"].map("{:,.2f}%".format)
dist_school_summary["% Overall Passing"] = dist_school_summary["% Overall Passing"].map("{:,.2f}%".format)

<a id='district_school_df'></a>
## Schools Summary Table


In [25]:
dist_school_summary

,School Type,Total Students,Total School Budget,Budget Per Student,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Bailey High School,District,4976,"$3,124,928",$628,77.05,81.03,66.68%,81.93%,54.64%
Cabrera High School,Charter,1858,"$1,081,356",$582,83.06,83.98,94.13%,97.04%,91.33%
Figueroa High School,District,2949,"$1,884,411",$639,76.71,81.16,65.99%,80.74%,53.20%
Ford High School,District,2739,"$1,763,916",$644,77.10,80.75,68.31%,79.30%,54.29%
Griffin High School,Charter,1468,"$917,500",$625,83.35,83.82,93.39%,97.14%,90.60%
Hernandez High School,District,4635,"$3,022,020",$652,77.29,80.93,66.75%,80.86%,53.53%
Holden High School,Charter,427,"$248,087",$581,83.80,83.81,92.51%,96.25%,89.23%
Huang High School,District,2917,"$1,910,635",$655,76.63,81.18,65.68%,81.32%,53.51%
Johnson High School,District,4761,"$3,094,650",$650,77.07,80.97,66.06%,81.22%,53.54%
Pena High School,Charter,962,"$585,858",$609,83.84,84.04,94.59%,95.95%,90.54%


<a id='top5_school_df'></a>
# Top 5 Schools based on Overall Score Percentage

The top 5 schools based on their overall perfomance, pulled from the District School Summary above and sorted by the 
`[% Overall Passing]` column.

Return to [Table of Contents](#table_of_contents)

In [26]:
top_performing = dist_school_summary.sort_values(["% Overall Passing"], axis=0, ascending=False)
top_performing.head(5)

,School Type,Total Students,Total School Budget,Budget Per Student,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Cabrera High School,Charter,1858,"$1,081,356",$582,83.06,83.98,94.13%,97.04%,91.33%
Thomas High School,Charter,1635,"$1,043,130",$638,83.42,83.85,93.27%,97.31%,90.95%
Griffin High School,Charter,1468,"$917,500",$625,83.35,83.82,93.39%,97.14%,90.60%
Wilson High School,Charter,2283,"$1,319,574",$578,83.27,83.99,93.87%,96.54%,90.58%
Pena High School,Charter,962,"$585,858",$609,83.84,84.04,94.59%,95.95%,90.54%


<a id='bottom5_school_df'></a>
# Bottom 5 Schools based on Overall Score Percentage

The bottom 5 schools based on their overall perfomance, pulled from the District School Summary above and sorted by the 
`[% Overall Passing]` column. 

Return to [Table of Contents](#table_of_contents)

In [27]:
bottom_performing = dist_school_summary.sort_values(["% Overall Passing"], axis=0, ascending=True)
bottom_performing.head(5)

,School Type,Total Students,Total School Budget,Budget Per Student,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Rodriguez High School,District,3999,"$2,547,363",$637,76.84,80.74,66.37%,80.22%,52.99%
Figueroa High School,District,2949,"$1,884,411",$639,76.71,81.16,65.99%,80.74%,53.20%
Huang High School,District,2917,"$1,910,635",$655,76.63,81.18,65.68%,81.32%,53.51%
Hernandez High School,District,4635,"$3,022,020",$652,77.29,80.93,66.75%,80.86%,53.53%
Johnson High School,District,4761,"$3,094,650",$650,77.07,80.97,66.06%,81.22%,53.54%


<a id='math_scores_summary'></a>
# Schools Math Scores by Grade

So, this area of code I personally think looks both nice and ugly. Nice because I think that it is all very legible, each block is seperated by grade and the merge cells create what I think is a nice indent between each block. Also, if I'm understanding correctly these blocks are applicable to any grading data in this format, so it is at least versatile. What I think is ugly is the fact that there are 4 seperate cells 9 lines long needing 3 merges to get to the final dataframe. Sometime later when I have the time and knowledge I want to come back and see if there's a better way to do this. I'm also going to refrain from commenting inside the cells themselves and will instead comment their function here, since they are all functionally the same; 

What each cell does is create a DataFrame from .loc of all the values .loc finds in `[Grade]` from the grade_scores dataframe that are exactly the same as (==) the specified grade, in the case of the first one its "9th". The columns are renamed because debugging would be kind of annoying if the headers of every cell were the same, and because when the final merged DataFrames ninth_tenth_m and eleventh_twelfth_m are merged if the columns aren't renamed the merge will not work and an error will occur. After the rename, the index is set as the school name and then grouped by the school name. The mean of the group is sent to the final DataFrame, which is what is the data seen in the final DataFrame.

Return to [Table of Contents](#table_of_contents)

In [28]:
grade_scores = district[["school_name","math_score","grade",]]

In [29]:
ninth_grade_mscores = grade_scores.loc[grade_scores['grade'] == "9th"]

In [30]:
ninth_grade_mscores = ninth_grade_mscores.rename(columns={"math_score":"9th Grade Average", "grade":"9th grade"})

ninth_grade_mscores = ninth_grade_mscores.set_index("school_name")

nine_grade_mgroup = ninth_grade_mscores.groupby(["school_name"])

ninth_grade_maverage = nine_grade_mgroup.mean()

In [31]:
tenth_grade_mscores = grade_scores.loc[grade_scores['grade'] == "10th"]

In [32]:
tenth_grade_mscores = tenth_grade_mscores.rename(columns={"math_score":"10th Grade Average", "grade":"10th grade"})

tenth_grade_mscores = tenth_grade_mscores.set_index("school_name")

tenth_grade_mgroup = tenth_grade_mscores.groupby(["school_name"])

tenth_grade_maverage = tenth_grade_mgroup.mean()

In [33]:
ninth_tenth_m = pd.merge(ninth_grade_maverage,tenth_grade_maverage,left_index=True, right_index=True)

In [34]:
eleventh_grade_mscores = grade_scores.loc[grade_scores['grade'] == "11th"]

In [35]:
eleventh_grade_mscores= eleventh_grade_mscores.rename(columns={"math_score":"11th Grade Average", "grade":"11th grade"})

eleventh_grade_mscores = eleventh_grade_mscores.set_index("school_name")

eleventh_grade_mgroup = eleventh_grade_mscores.groupby(["school_name"])

eleventh_grade_maverage = eleventh_grade_mgroup.mean()

In [36]:
twelfth_grade_mscores = grade_scores.loc[grade_scores['grade'] == "12th"]

In [37]:
twelfth_grade_mscores= twelfth_grade_mscores.rename(columns={"math_score":"12th Grade Average", "grade":"12th grade"})

twelfth_grade_mscores = twelfth_grade_mscores.set_index("school_name")

twelfth_grade_mgroup = twelfth_grade_mscores.groupby(["school_name"])

twelfth_grade_maverage = twelfth_grade_mgroup.mean()

In [38]:
eleventh_twelfth_m = pd.merge(eleventh_grade_maverage,twelfth_grade_maverage,left_index=True, right_index=True)

In [39]:
average_math_by_grade = pd.merge(ninth_tenth_m,eleventh_twelfth_m,left_index=True, right_index=True)
average_math_by_grade.index.name = None

<a id='math_scores_df'></a>
## Math Scores By Grade

In [40]:
average_math_by_grade

,9th Grade Average,10th Grade Average,11th Grade Average,12th Grade Average
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248
Pena High School,83.625455,83.372000,84.328125,84.121547


<a id='reading_scores_summary'></a>
# Schools Reading Scores by Grade

For comments, see what I wrote under [Schools Math Scores by Grade](#math_scores_summary). These cells are functionally the same just with the parameter of reading scores instead of math scores.

Return to [Table of Contents](#table_of_contents)

In [41]:
grade_scores = district[["school_name","reading_score","grade",]]

In [42]:
ninth_grade_rscores = grade_scores.loc[grade_scores['grade'] == "9th"]

ninth_grade_rscores = ninth_grade_rscores.rename(columns={"reading_score":"9th Grade Average", "grade":"9th grade"})

ninth_grade_rscores = ninth_grade_rscores.set_index("school_name")

nine_grade_rgroup = ninth_grade_rscores.groupby(["school_name"])

ninth_grade_raverage = nine_grade_rgroup.mean()

In [43]:
ninth_grade_rscores = grade_scores.loc[grade_scores['grade'] == "9th"]

ninth_grade_rscores = ninth_grade_rscores.rename(columns={"reading_score":"9th Grade Average", "grade":"9th grade"})

ninth_grade_rscores = ninth_grade_rscores.set_index("school_name")

nine_grade_rgroup = ninth_grade_rscores.groupby(["school_name"])

ninth_grade_raverage = nine_grade_rgroup.mean()

In [44]:
tenth_grade_rscores = grade_scores.loc[grade_scores['grade'] == "10th"]

tenth_grade_rscores = tenth_grade_rscores.rename(columns={"reading_score":"10th Grade Average", "grade":"10th grade"})

tenth_grade_rscores = tenth_grade_rscores.set_index("school_name")

tenth_grade_rgroup = tenth_grade_rscores.groupby(["school_name"])

tenth_grade_raverage = tenth_grade_rgroup.mean()

In [45]:
ninth_tenth_r = pd.merge(ninth_grade_raverage,tenth_grade_raverage,left_index=True, right_index=True)

In [46]:
eleventh_grade_rscores = grade_scores.loc[grade_scores['grade'] == "11th"]

eleventh_grade_rscores = eleventh_grade_rscores.rename(columns={"reading_score":"11th Grade Average", "grade":"11th grade"})

eleventh_grade_rscores = eleventh_grade_rscores.set_index("school_name")

eleventh_grade_rgroup = eleventh_grade_rscores.groupby(["school_name"])

eleventh_grade_raverage = eleventh_grade_rgroup.mean()

In [47]:
twelfth_grade_rscores = grade_scores.loc[grade_scores['grade'] == "12th"]

twelfth_grade_rscores = twelfth_grade_rscores.rename(columns={"reading_score":"12th Grade Average", "grade":"12th grade"})

twelfth_grade_rscores = twelfth_grade_rscores.set_index("school_name")

twelfth_grade_rgroup = twelfth_grade_rscores.groupby(["school_name"])

twelfth_grade_raverage = twelfth_grade_rgroup.mean()

In [48]:
eleventh_twelfth_r = pd.merge(eleventh_grade_raverage,twelfth_grade_raverage,left_index=True, right_index=True)

In [49]:
average_reading_by_grade = pd.merge(ninth_tenth_r,eleventh_twelfth_r,left_index=True, right_index=True)
average_reading_by_grade.index.name = None

<a id='reading_scores_df'></a>
## Reading Scores by Grade

In [50]:
average_reading_by_grade

,9th Grade Average,10th Grade Average,11th Grade Average,12th Grade Average
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564
Pena High School,83.807273,83.612000,84.335938,84.591160


<a id='spending_size_type_summary'></a>
# Summaries on Spending, School Size, and Type 

The next two DataFrames utilize binning methods to organize their data. The first being on budget, the second on school size.

Return to [Table of Contents](#table_of_contents)

In [51]:
#The first summary is for the average scores based on the spending range per student. 
# More to the point, the average test scores per average dollar spent per student. 

In [52]:
#This is what I used to get an idea of how to build the bins. 
#district_school_summary["Budget Per Student"].sort_values(ascending=True)

In [53]:
#The range I considered at first was a $15 dollar range which I thought in terms of a student could be the difference
# between a school provided lunch or not. I settled on the current scale since it's broken into three three groups of 
# four and one group of 3 (4,4,4,3) which is concise enough for what I need. 

budget_bins = [0,585,630,645,680]
budget_labels = ["<$585","$585-630","$630-645","$645-680"]

In [54]:
budget_summary =  district_school_summary[ ["math_score","reading_score","% Passing Math","% Passing Reading",
                                                  "% Overall Passing"] ]

budget_summary["Budget Labels"] = pd.cut(district_school_summary["Budget Per Student"], bins=budget_bins, labels=budget_labels)

In [55]:
#Same trend as above, groupby then.mean()

budget_summary_group = budget_summary.groupby(["Budget Labels"])
budget_summary_final = budget_summary_group.mean()

In [56]:
#And then formatting. Rinse repeat for each summary. I also add a counts per observed value because I felt that 
# information was too important to leave off.

budget_summary_final["Schools Per Range"] = budget_summary["Budget Labels"].value_counts()
budget_summary_final = budget_summary_final.round(decimals=2)
budget_summary_final = budget_summary_final.rename(columns={
    "math_score":"Average Math Score", 
    "reading_score":"Average Reading Score"})
budget_summary_final.index.name = "Average Spending Per Student"

<a id='spending_type_size_df'></a>
###### Average Scores by the Average Spent per Student

In [57]:
budget_summary_final

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing,Schools Per Range
Average Spending Per Student,,,,,,
<$585,83.46,83.93,93.46,96.61,90.37,4
$585-630,81.90,83.16,87.13,92.72,81.42,4
$630-645,78.52,81.62,73.48,84.39,62.86,4
$645-680,77.00,81.03,66.16,81.13,53.53,3


In [58]:
#Code for getting the size summary begins.

In [59]:
size_bins = [0,1000,2000,5000]
size_labels = ["Small (<1000)","Medium (1000-2000)","Large (2000-5000)"]

In [60]:
size_summary =  district_school_summary[ ["math_score","reading_score","% Passing Math","% Passing Reading",
                                                  "% Overall Passing"] ]
size_summary["size Labels"] = pd.cut(district_school_summary["size"], bins=size_bins, labels=size_labels)

In [61]:
size_summary_group = size_summary.groupby(["size Labels"])
size_summary_final = size_summary_group.mean()

In [62]:
size_summary_final["Schools Per Range"] = size_summary["size Labels"].value_counts()
size_summary_final = size_summary_final.round(decimals=2)
size_summary_final = size_summary_final.rename(columns={
    "math_score":"Average Math Score", 
    "reading_score":"Average Reading Score"})
size_summary_final.index.name = "School Size"

##### Average Scores by Student Population 

In [63]:
size_summary_final

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing,Schools Per Range
School Size,,,,,,
Small (<1000),83.82,83.93,93.55,96.10,89.88,2
Medium (1000-2000),83.37,83.86,93.60,96.79,90.62,5
Large (2000-5000),77.75,81.34,69.96,82.77,58.29,8


In [64]:
#Code for getting the size summary begins. No bins for this, just groupby([type]).

In [65]:
type_summary = district_school_summary[ ["math_score","reading_score","% Passing Math","% Passing Reading",
                                                  "% Overall Passing", "type"] ]

In [66]:
type_summary_group = type_summary.groupby(["type"])
type_summary_final = type_summary_group.mean()

In [67]:
type_summary_final["Schools Per Range"] = type_summary["type"].value_counts()
type_summary_final = type_summary_final.round(decimals=2)
type_summary_final = type_summary_final.rename(columns={
    "math_score":"Average Math Score", 
    "reading_score":"Average Reading Score"})
type_summary_final.index.name = "School Type"

##### Average Scores by School Type 

In [68]:
type_summary_final

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing,Schools Per Range
School Type,,,,,,
Charter,83.47,83.90,93.62,96.59,90.43,8
District,76.96,80.97,66.55,80.80,53.67,7


<a id='table_of_contents'></a>
# Table Of Contents

Notebook Setup will take you back to the top. Summary links will take you to the start of the code for a given section. Written below this table of contents are some trends I observed from the data analysed. All the DataFrames are presented below as well for clearer side by side viewing with my thoughts on each one.

[Notebook Setup](#notebook_setup)

[District Summary](#district_summary)

[School Summary](#school_summary)

[Top 5 Schools](#top5_school_df)

[Bottom 5 Schools](#bottom5_school_df)

[Math Scores by Grade Summary](#math_scores_summary)

[Reading Scores by Grade Summary](#reading_scores_summary)

[Spending, School Size, and Type Summary](#spending_size_type_summary)

[Bottom of the Notebook](#bottom_of_notebook)

This is my final report and thoughts based on my reports found under each table below. To summarize, Charter schools far outscore District schools in every recorded area, with the closest being a difference of 2.93 in Average Reading Scores in favor of Charter Schools. Charter schools are also smaller than every school District schools, with the smallest difference in students being 456 students between Ford High School(2739) and Wilson High School(2283) and the largest difference being 4,549 students between Bailey High School(4976) and Holden High School (427). The Total School budget also seems to scale based on total student population. 

My thoughts are that the most insightful information would probably come from comparisons of Ford High School and Wilson High School, who have a difference of 456 students, a difference that is negligible when compared to the consistency in scores that the respective high schools share with schools of the same type with a similar or greater difference in students. Ford High School also has budget $444,342 greater than that of Wilson, all the while Wilson has the lowest budget per student district wide and is the 4th highest overall performer. 

The most interesting observation I see between these two schools is the drasticity in their comparative scores, and how little a difference the observations made above matter in the consistency in the scores of related school types. 89-90% overall passing in charter schools across the entirety of charter schools, 52-54% overall passing in district schools across the entirtey of district schools. I'll finish my thoughts with this final question - What is Wilson High School doing to the scale of Ford High School that the rest of the Charter schools are doing that is not happening at the District schools?  

# District Summary

In [69]:
dist_summary_final

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,15,"39,170","$24,649,428",78.99,81.88,74.98%,85.81%,65.17%


To someone with no background in profiling school districts, an overall passing % of 65% seems worrying. 

# School Summary

In [70]:
dist_school_summary

,School Type,Total Students,Total School Budget,Budget Per Student,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Bailey High School,District,4976,"$3,124,928",$628,77.05,81.03,66.68%,81.93%,54.64%
Cabrera High School,Charter,1858,"$1,081,356",$582,83.06,83.98,94.13%,97.04%,91.33%
Figueroa High School,District,2949,"$1,884,411",$639,76.71,81.16,65.99%,80.74%,53.20%
Ford High School,District,2739,"$1,763,916",$644,77.10,80.75,68.31%,79.30%,54.29%
Griffin High School,Charter,1468,"$917,500",$625,83.35,83.82,93.39%,97.14%,90.60%
Hernandez High School,District,4635,"$3,022,020",$652,77.29,80.93,66.75%,80.86%,53.53%
Holden High School,Charter,427,"$248,087",$581,83.80,83.81,92.51%,96.25%,89.23%
Huang High School,District,2917,"$1,910,635",$655,76.63,81.18,65.68%,81.32%,53.51%
Johnson High School,District,4761,"$3,094,650",$650,77.07,80.97,66.06%,81.22%,53.54%
Pena High School,Charter,962,"$585,858",$609,83.84,84.04,94.59%,95.95%,90.54%


I have few comments on this table as an entirety, it provides a nice reference for when I would want to single out a single school to check its records, and I think would show the most worth as a sorting tool, and in the future a comparison over time tool. 

# Highest-Performing Schools (by % Overall Passing)

In [71]:
top_performing.head(5)

,School Type,Total Students,Total School Budget,Budget Per Student,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Cabrera High School,Charter,1858,"$1,081,356",$582,83.06,83.98,94.13%,97.04%,91.33%
Thomas High School,Charter,1635,"$1,043,130",$638,83.42,83.85,93.27%,97.31%,90.95%
Griffin High School,Charter,1468,"$917,500",$625,83.35,83.82,93.39%,97.14%,90.60%
Wilson High School,Charter,2283,"$1,319,574",$578,83.27,83.99,93.87%,96.54%,90.58%
Pena High School,Charter,962,"$585,858",$609,83.84,84.04,94.59%,95.95%,90.54%


Interestingly, all of the top performing schools are charter schools. I also believe Cabrera High School has the 3rd lowest spending per student as well, with Wilson High School being the overall lowest spender per student. 

# Lowest-Performing Schools (by % Overall Passing)

In [72]:
bottom_performing.head(5)

,School Type,Total Students,Total School Budget,Budget Per Student,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Rodriguez High School,District,3999,"$2,547,363",$637,76.84,80.74,66.37%,80.22%,52.99%
Figueroa High School,District,2949,"$1,884,411",$639,76.71,81.16,65.99%,80.74%,53.20%
Huang High School,District,2917,"$1,910,635",$655,76.63,81.18,65.68%,81.32%,53.51%
Hernandez High School,District,4635,"$3,022,020",$652,77.29,80.93,66.75%,80.86%,53.53%
Johnson High School,District,4761,"$3,094,650",$650,77.07,80.97,66.06%,81.22%,53.54%


Just as all the top performers were Charter schools, all the bottom performing schools are district schools. Opposite as well to the Charter Schools, the District schools also have a much higher budget per student. Having looked at this table though, the student populations of the school types are not comparable. 

# Math Scores by Grade

In [73]:
average_math_by_grade

,9th Grade Average,10th Grade Average,11th Grade Average,12th Grade Average
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248
Pena High School,83.625455,83.372000,84.328125,84.121547


If it weren't for Holden High Schools weird looking 11th grade Average score of an unrounded perfect 85.000000 I would say the results are otherwise unremarkable, showing little to no change across each respective schools grades. The scores range from high 70s to low 80s. 

# Reading Scores by Grade

In [74]:
average_reading_by_grade

,9th Grade Average,10th Grade Average,11th Grade Average,12th Grade Average
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564
Pena High School,83.807273,83.612000,84.335938,84.591160


Just like above, there's little to no change as each grade progressess. The scores are all consistently in the range of the low 80s. 

# Scores by School Spending

In [75]:
budget_summary_final

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing,Schools Per Range
Average Spending Per Student,,,,,,
<$585,83.46,83.93,93.46,96.61,90.37,4
$585-630,81.90,83.16,87.13,92.72,81.42,4
$630-645,78.52,81.62,73.48,84.39,62.86,4
$645-680,77.00,81.03,66.16,81.13,53.53,3


The less spent on students the better the scores. Referencing the School Summary table we can find that every school in that range is a charter school. Also to note, the bottom 3 performing schools are also the same 3 schools found in the $645-$680 range. 

# Scores by School Size

In [76]:
size_summary_final

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing,Schools Per Range
School Size,,,,,,
Small (<1000),83.82,83.93,93.55,96.10,89.88,2
Medium (1000-2000),83.37,83.86,93.60,96.79,90.62,5
Large (2000-5000),77.75,81.34,69.96,82.77,58.29,8


The 2 in the small range are both charters, though only 1 is a top performer. Of the 5 schools in the Medium range, 3 are top performers, and the other 2 are both the next runner ups for top performing. In large are all of the bottom performing schools and the last top performing school. 

# Scores by School Type

In [77]:
type_summary_final

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing,Schools Per Range
School Type,,,,,,
Charter,83.47,83.90,93.62,96.59,90.43,8
District,76.96,80.97,66.55,80.80,53.67,7


This is one of the most considerable observations that can be made, and the one that I've been pointing out in pretty much every report. Charter schools far out score District schools in their percentage of overall passing students. 

<a id='bottom_of_notebook'></a>
---
[Click here to go to the start of the Notebook.](#notebook_setup)

[Click here to go to the Table of Contents](#table_of_contents)

---
> Notebook written by Jacob McManaman.

> Data generated by Mockaroo, LLC. (2021) Realistic Data Generator. https://www.mockaroo.com/. Modified by Trilogy Education Services, LLC.